In [1]:
from datetime import date, timedelta
import os
import pickle
import sys
import pandas as pd
import numpy as np
sys.path.append("..")

from abautomator.visualizer import BasicVisualizer, AbsDiffVisualizer, RelDiffVisualizer

from bokeh.io import output_notebook
from bokeh.plotting import show
from bokeh.models import MultiChoice, CustomJS
from bokeh.layouts import column, row

output_notebook()

Loading BokehJS ...

In [2]:
# %pip install -U pandas

In [3]:
# name = "rollback_analy"
name = "inspiration_analy"
analy = pickle.load(
    open(os.path.join("..", "tests", "cache", f"{name}.p"), "rb" )
)

In [4]:
df = analy.get_rel_diff_confidence_intervals()
df.head()

exp_cond             metric   tx_mean    tx_std  tx_count  ctrl_mean  \
0     Video01    n_entered_phone  0.508623  0.500202   37053.0   0.510671   
1     Video01  pct_entered_phone  0.508488  0.499935   37053.0   0.510563   
2  Carousel01    n_entered_phone  0.510820  0.500162   36784.0   0.510671   
3  Carousel01  pct_entered_phone  0.510684  0.499893   36784.0   0.510563   
4  Carousel02    n_entered_phone  0.507323  0.500333   36801.0   0.510671   

   ctrl_std  ctrl_count  abs_mean   abs_std   p_value  upper_68_ci  \
0  0.500110     36828.0 -0.002048  0.003680  0.577794     0.319531   
1  0.499895     36828.0 -0.002075  0.001839  0.129640    -0.046143   
2  0.500110     36828.0  0.000149  0.003687  0.967829     0.751059   
3  0.499895     36828.0  0.000121  0.001842  0.473737     0.384642   
4  0.500110     36828.0 -0.003348  0.003687  0.363838     0.066364   

   lower_68_ci  upper_95_ci  lower_95_ci      mean       std  
0    -1.121787     1.011364    -1.813620 -0.401128  0.720659  
1    -0.766597     0.299674    -1.112414 -0.406370  0.360227  
2    -0.692825     1.444124    -1.385889  0.029117  0.721942  
3    -0.337095     0.731075    -0.683528  0.023773  0.360868  
4    -1.377602     0.759467    -2.070705 -0.655619  0.721983

In [21]:
from abautomator.visualizer import utils, btn, bars

metric_order = list(df["metric"].unique())
cond_order = list(df["exp_cond"].unique())

metric_order = {item: i for i, item in enumerate(metric_order)}
cond_order = {item: i for i, item in enumerate(cond_order)}
df["metric_order"] = df['metric'].map(metric_order)
df["cond_order"] = df['exp_cond'].map(cond_order)
df = df.sort_values(by=['metric_order', 'cond_order'], ascending=(False, False))

source = utils.convert_df_to_source(df)

In [22]:
print(raw_cond_order)
print(COND_OPTIONS)

['Video01', 'Carousel01', 'Carousel02', 'Carousel03', 'Carousel04']
['Video01', 'Carousel01', 'Carousel02', 'Carousel03', 'Carousel04']


In [29]:
def get_cond_multichoice(y_range, source, cond_options):
    result = MultiChoice(value=cond_options, options=cond_options)
    callback = CustomJS(
        args=dict(yRange=y_range, source=source),
        code="""
        // Filter y-axis labels
        const multiChoiceArray = this
        let tempList = source.data["factor_label"].filter(function (currentElement) {
          return multiChoiceArray.value.includes(currentElement[1]);
        });
        yRange.factors = tempList;
        """
    )
    result.js_on_change("value", callback)
    return result

In [28]:
def get_metric_multichoice(y_range, source, metric_options):
    result = MultiChoice(value=metric_options, options=metric_options)
    callback = CustomJS(
        args=dict(yRange=y_range, source=source),
        code="""
        // Filter y-axis labels
        const multiChoiceArray = this
        let tempList = source.data["factor_label"].filter(function (currentElement) {
          return multiChoiceArray.value.includes(currentElement[0]);
        });
        yRange.factors = tempList;
        """
    )
    result.js_on_change("value", callback)
    return result

In [34]:
METRIC_OPTIONS = utils.get_cleaned_metrics(raw_metric_order)
COND_OPTIONS = raw_cond_order

cat_order = [(metric, cond) for metric in reversed(METRIC_OPTIONS) for cond in reversed(COND_OPTIONS)]

vis = RelDiffVisualizer(cat_order, source)
fig = vis.get_figure()

show(
    column(
        fig,
        row(get_cond_multichoice(fig.y_range, source, COND_OPTIONS), get_metric_multichoice(fig.y_range, source, METRIC_OPTIONS)),
        btn.get_stat_sig_btn(fig.renderers),
    )
)
# show(fig)

In [50]:
COND_OPTIONS

['Carousel01', 'Carousel02', 'Carousel03', 'Carousel04', 'Video01']

In [51]:
source.data["exp_cond"]

array(['Carousel04', 'Carousel03', 'Carousel02', 'Carousel01', 'Video01',
       'Carousel04', 'Carousel03', 'Carousel02', 'Carousel01', 'Video01',
       'Carousel04', 'Carousel03', 'Carousel02', 'Carousel01', 'Video01',
       'Carousel04', 'Carousel03', 'Carousel02', 'Carousel01', 'Video01'],
      dtype=object)

In [29]:
show(fig)

In [28]:
from bokeh.models import Segment, HBar
len([renderer for renderer in fig.renderers if isinstance(renderer.glyph, HBar)])

8

In [15]:
len(fig.renderers)

30

In [ ]:
for item in fig.renderers if item_pro

In [51]:
    for (const item of legend.items){
        if (multiChoiceArray.value.includes(item.label["value"])) {
            item.visible = true;
        } else {
            item.visible = false;
        }
        console.log(item.label);
        console.log(item.visible);
    }

[Legend(id='8519', ...), Legend(id='8519', ...)]

In [30]:
for item in fig.legend[0].items:
    print(item.label)

{'value': 'Carousel01'}
{'value': 'Carousel02'}
{'value': 'Carousel03'}
{'value': 'Carousel04'}
{'value': 'Video01'}


In [22]:
fig.__dict__["_property_values"]["center"][2].__dict__

{'_id': '1659',
 '_document': <bokeh.document.document.Document at 0x7f36e0711880>,
 '_temp_document': None,
 '_event_callbacks': {},
 '_callbacks': {},
 '_property_values': {'items': [LegendItem(id='1660', ...),
   LegendItem(id='1691', ...),
   LegendItem(id='1722', ...),
   LegendItem(id='1753', ...),
   LegendItem(id='1784', ...)],
  'click_policy': 'mute'},
 '_unstable_default_values': {'js_property_callbacks': {},
  'js_event_callbacks': {},
  'tags': [],
  'subscribed_events': []},
 '_unstable_themed_values': {},
 '_initialized': True}

In [18]:
from abautomator.visualizer import utils
source = utils.convert_df_to_source(vis.df)

In [5]:
class TestVisualizer(RelDiffVisualizer):
    """ Parent object. Not to be initiated directly """

    def __init__(self, df, x_axis_label):
        self.df = df
        self.x_axis_label = x_axis_label

    def get_figure(self):
        df = self.df.copy()
        df = _clean_exp_conds(df)
        source = ColumnDataSource(df)

        fig = _init_figure(source, self._get_tool_tips())

        _add_zero_span(fig)

        _add_bars(fig, source)

        # _set_legend(fig)
        _set_x_axis(fig, self.x_axis_label)
        _set_y_axis(fig)
        
        return fig

In [6]:
from abautomator.visualizer.parent import _cond_to_color_mapper, add_error_bars, add_core_interval

def _add_sig_bars(fig, source):

    for exp_cond, color in _cond_to_color_mapper(source).items():
        bools = [
            True if p_value <= 0.05 else False
            for p_value in source.data['p_value']
        ]
        view = CDSView(source=source, filters=[BooleanFilter(bools)])

        fig_core = (fig, source, view)

#         lower_eb, upper_eb = add_error_bars(fig_core)
        core_interval = add_core_interval(fig, exp_cond, "blue", fig_core, add_legend=False)
        core_interval.tags = ["stat_sig"]
        core_interval.visible = False

#         core_interval.js_link('visible', lower_eb, 'visible')
#         core_interval.js_link('visible', upper_eb, 'visible')

ImportError: cannot import name '_cond_to_color_mapper' from 'abautomator.visualizer.parent' (../abautomator/visualizer/parent.py)

In [7]:
vis = RelDiffVisualizer(analy.get_rel_diff_confidence_intervals())
fig = vis.get_figure()

from bokeh.layouts import column
from bokeh.models import Button, CustomJS, ColumnDataSource, CDSView, BooleanFilter

button = Button(label="Toggle Stat Sig", button_type="danger")
callback = CustomJS(
    args=dict(
        glyphs=fig.renderers,
        stat_sig=stat_sig_visible
    ),
    code="""
    for (const glyph of glyphs){
        if (glyph.tags.includes("stat_sig")) {
            var stat_sig_visible = glyph.visible;
        }
    }
    console.log("SIG");
    console.log(stat_sig_visible);
    console.log("SIG");
    
    if (stat_sig_visible) {
        var muted = false;
    } else {
        var muted = true;
    }

    for (const glyph of glyphs){
        if (glyph.tags.includes("stat_sig")) {
            glyph.visible = !glyph.visible;
        } else {
            glyph.muted = muted;
        }
    }
    """
    )

button.js_on_click(callback)

# fig.legend.click_policy="mute"
# fig.add_layout(fig.legend[0], 'right')
# fig = _add_sig_bars(fig, ColumnDataSource(analy.get_rel_diff_confidence_intervals()))

show(column(fig, button))
show(fig)

AttributeError: 'Column' object has no attribute 'renderers'

In [16]:
fig.renderers[2]._property_values

{'glyph': HBar(id='2464', ...),
 'nonselection_glyph': HBar(id='2465', ...),
 'view': CDSView(id='2454', ...),
 'data_source': ColumnDataSource(id='2425', ...),
 'js_property_callbacks': {'change:visible': [CustomJS(id='2467', ...),
   CustomJS(id='2468', ...)]},
 'visible': False,
 'tags': ['stat_sig']}

In [8]:
vis = BasicVisualizer(analy.get_basic_confidence_intervals())
fig = vis.get_figure()
# fig.add_layout(fig.legend[1], 'right')
show(fig)

In [6]:
vis = AbsDiffVisualizer(analy.get_abs_diff_confidence_intervals())
show(vis.get_figure())

Carousel01 [False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False]
Carousel02 [False, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False]
Carousel03 [False, False, False, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False]
Carousel04 [False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, True, True]
Video01 [True, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False]


Carousel01 [False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False]
Carousel02 [False, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False]
Carousel03 [False, False, False, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False]
Carousel04 [False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, True, True]
Video01 [True, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False]


In [9]:
analy.get_rel_diff_confidence_intervals()

exp_cond                metric   tx_mean    tx_std  tx_count  ctrl_mean  \
0      Video01       n_entered_phone  0.508623  0.500202   37053.0   0.510671   
1      Video01     pct_entered_phone  0.508488  0.499935   37053.0   0.510563   
2   Carousel02       n_entered_phone  0.507323  0.500333   36801.0   0.510671   
3   Carousel02     pct_entered_phone  0.507133  0.499956   36801.0   0.510563   
4   Carousel03       n_entered_phone  0.512655  0.500392   36666.0   0.510671   
5   Carousel03     pct_entered_phone  0.512382  0.499853   36666.0   0.510563   
6   Carousel01       n_entered_phone  0.510820  0.500162   36784.0   0.510671   
7   Carousel01     pct_entered_phone  0.510684  0.499893   36784.0   0.510563   
8   Carousel04       n_entered_phone  0.506039  0.500188   36681.0   0.510671   
9   Carousel04     pct_entered_phone  0.505930  0.499972   36681.0   0.510563   
10     Video01    n_granted_contacts  0.188379  0.391020   37053.0   0.191349   
11     Video01  pct_granted_contacts  0.188379  0.391020   37053.0   0.191322   
12  Carousel02    n_granted_contacts  0.190973  0.393281   36801.0   0.191349   
13  Carousel02  pct_granted_contacts  0.190892  0.393009   36801.0   0.191322   
14  Carousel03    n_granted_contacts  0.195658  0.396918   36666.0   0.191349   
15  Carousel03  pct_granted_contacts  0.195576  0.396649   36666.0   0.191322   
16  Carousel01    n_granted_contacts  0.196064  0.397023   36784.0   0.191349   
17  Carousel01  pct_granted_contacts  0.196064  0.397023   36784.0   0.191322   
18  Carousel04    n_granted_contacts  0.190180  0.392587   36681.0   0.191349   
19  Carousel04  pct_granted_contacts  0.190126  0.392406   36681.0   0.191322   

    ctrl_std  ctrl_count  abs_mean   abs_std  upper_68_ci  lower_68_ci  \
0   0.500110     36828.0 -0.002048  0.003680     0.319531    -1.121787   
1   0.499895     36828.0 -0.002075  0.001839    -0.046143    -0.766597   
2   0.500110     36828.0 -0.003348  0.003687     0.066364    -1.377602   
3   0.499895     36828.0 -0.003430  0.001842    -0.310889    -1.032592   
4   0.500110     36828.0  0.001984  0.003691     1.111109    -0.334269   
5   0.499895     36828.0  0.001819  0.001844     0.717502    -0.004787   
6   0.500110     36828.0  0.000149  0.003687     0.751059    -0.692825   
7   0.499895     36828.0  0.000121  0.001842     0.384642    -0.337095   
8   0.500110     36828.0 -0.004633  0.003689    -0.184706    -1.629643   
9   0.499895     36828.0 -0.004633  0.001844    -0.546300    -1.268606   
10  0.393437     36828.0 -0.002970  0.002886    -0.043962    -3.060486   
11  0.393347     36828.0 -0.002943  0.001443    -0.784109    -2.292395   
12  0.393437     36828.0 -0.000376  0.002899     1.318747    -1.711644   
13  0.393347     36828.0 -0.000430  0.001449     0.532454    -0.982239   
14  0.393437     36828.0  0.004309  0.002915     3.775576     0.728372   
15  0.393347     36828.0  0.004254  0.001457     2.985282     1.462160   
16  0.393437     36828.0  0.004715  0.002913     3.986429     0.941250   
17  0.393347     36828.0  0.004742  0.001457     3.239701     1.717063   
18  0.393437     36828.0 -0.001169  0.002899     0.904296    -2.125909   
19  0.393347     36828.0 -0.001196  0.001449     0.132187    -1.382585   

    upper_95_ci  lower_95_ci      mean       std  
0      1.040191    -1.842447 -0.401128  0.720659  
1      0.314083    -1.126823 -0.406370  0.360227  
2      0.788346    -2.099584 -0.655619  0.721983  
3      0.049962    -1.393444 -0.671741  0.360852  
4      1.833798    -1.056959  0.388420  0.722689  
5      1.078646    -0.365931  0.356357  0.361144  
6      1.473001    -1.414767  0.029117  0.721942  
7      0.745510    -0.697963  0.023773  0.360868  
8      0.537762    -2.352111 -0.907175  0.722468  
9     -0.185146    -1.629759 -0.907453  0.361153  
10     1.464299    -4.568747 -1.552224  1.508262  
11    -0.029967    -3.046537 -1.538252  0.754143  
12     2.833942    -3.226839 -0.196449  1.515195  
13     1.289801    -1.739586 -0.224

In [8]:
# df["t_calc"] = df["mean"]/df["std"]
# df["t_crit"] = scipy.stats.t.ppf(q=1-.05/2, df=df["tx_count"] + df["ctrl_count"] - 2)
# df["p_value"] = scipy.stats.t.sf(np.abs(df["t_calc"]), df["tx_count"] + df["ctrl_count"] - 2)*2  # two-sided pvalue = Prob(abs(t)>tt)
# df.head()